<a href="https://colab.research.google.com/github/christiannofreitas/Mining-the-Social-Web-2nd-Edition/blob/master/Analisador%20de%20video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das dependências
!pip install yt-dlp pydub SpeechRecognition ffmpeg-python nltk
!apt install ffmpeg -qq

import os
import yt_dlp
from pydub import AudioSegment
import speech_recognition as sr
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Download dos recursos do NLTK (com tratamento de erro)
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab')  # Específico para português
except Exception as e:
    print(f"Erro ao baixar recursos do NLTK: {e}")
    print("Tentando continuar com recursos básicos...")

def download_youtube_audio(video_url, output_dir='.'):
    """Baixa áudio de vídeo do YouTube"""
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'quiet': True,
        'extract_flat': True,
        'force-overwrites': True,
        'no-check-certificate': True
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(video_url, download=True)
            original_filename = ydl.prepare_filename(info_dict)
            audio_filename = original_filename.replace('.webm', '.wav').replace('.m4a', '.wav')

            if not os.path.exists(audio_filename):
                if os.path.exists(original_filename):
                    audio_filename = original_filename
                else:
                    raise FileNotFoundError("Arquivo de áudio não foi baixado corretamente")

            video_title = info_dict.get('title', 'audio').replace('/', '-')
            return audio_filename, video_title
    except Exception as e:
        raise Exception(f"Erro ao baixar o áudio: {str(e)}")

def transcribe_audio(audio_path, chunk_size=30):
    """Transcreve áudio para texto"""
    recognizer = sr.Recognizer()

    try:
        audio = AudioSegment.from_file(audio_path).normalize()
    except Exception as e:
        raise Exception(f"Erro ao carregar áudio: {str(e)}")

    chunks = [audio[i*1000*chunk_size:(i+1)*1000*chunk_size]
              for i in range(len(audio)//(1000*chunk_size)+1)]

    full_text = ""

    for i, chunk in enumerate(chunks):
        temp_file = f"temp_{i}.wav"
        try:
            chunk.export(temp_file, format="wav")
            with sr.AudioFile(temp_file) as source:
                audio_data = recognizer.record(source)
                try:
                    text = recognizer.recognize_google(audio_data, language='pt-BR')
                    full_text += text + " "
                    print(f"Chunk {i+1}/{len(chunks)} transcrito")
                except sr.UnknownValueError:
                    print(f"Chunk {i+1} não reconhecido")
        finally:
            if os.path.exists(temp_file):
                os.remove(temp_file)

    return full_text.strip()

def generate_summary(text, num_sentences=5):
    """Gera resumo automático"""
    try:
        # Tenta português, fallback para inglês
        try:
            sentences = sent_tokenize(text, language='portuguese')
            stop_words = set(stopwords.words('portuguese'))
        except:
            sentences = sent_tokenize(text)
            stop_words = set(stopwords.words('english'))

        if len(sentences) <= num_sentences:
            return "\n".join(sentences)

        words = [word for word in word_tokenize(text.lower())
                if word.isalnum() and word not in stop_words]

        freq_table = {}
        for word in words:
            freq_table[word] = freq_table.get(word, 0) + 1

        sentence_scores = {}
        for sentence in sentences:
            for word in word_tokenize(sentence.lower()):
                if word in freq_table:
                    sentence_scores[sentence] = sentence_scores.get(sentence, 0) + freq_table[word]

        top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
        return "\n".join(top_sentences)

    except Exception as e:
        print(f"Erro ao gerar resumo: {e}")
        return "\n".join(sentences[:num_sentences])

def save_results(transcription, video_title):
    """Salva transcrição e resumo em arquivos TXT"""
    try:
        base_name = f"transcricao_{video_title[:50]}".replace(" ", "_")
        transcript_file = f"{base_name}_completo.txt"
        summary_file = f"{base_name}_resumo.txt"

        # Salva transcrição completa
        with open(transcript_file, 'w', encoding='utf-8') as f:
            f.write(f"Título: {video_title}\n\n")
            f.write(transcription)

        # Gera e salva resumo
        summary = generate_summary(transcription)
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write(f"Título: {video_title}\n\n")
            f.write("=== RESUMO ===\n\n")
            f.write(summary)
            f.write("\n\n=== FIM DA TRANSCRIÇÃO ===")

        print(f"\nArquivos salvos:")
        print(f"- Transcrição completa: {transcript_file}")
        print(f"- Resumo: {summary_file}")
        return True

    except Exception as e:
        print(f"\nErro ao salvar arquivos: {e}")
        return False

def main():
    video_url = input("Cole a URL do vídeo do YouTube: ").strip()

    if not video_url.startswith(('http://', 'https://')):
        print("URL inválida. Insira uma URL válida do YouTube.")
        return

    try:
        print("\n1. Baixando áudio...")
        audio_file, video_title = download_youtube_audio(video_url)

        print("\n2. Transcrevendo áudio...")
        transcription = transcribe_audio(audio_file)

        print("\n3. Salvando resultados...")
        save_results(transcription, video_title)

        if os.path.exists(audio_file):
            os.remove(audio_file)

        print("\n✅ Processo concluído com sucesso!")

    except Exception as e:
        print(f"\n❌ Erro: {e}")
        print("Possíveis soluções:")
        print("- Verifique a URL e conexão com a internet")
        print("- Tente um vídeo mais curto")
        print("- Verifique os recursos do NLTK")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.9 MB/s eta 0:00:00
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Cole a URL do vídeo do YouTube: https://www.youtube.com/live/B1pREozQDgw?si=oxvy0QeLh2twravU

1. Baixando áudio...

2. Transcrevendo áudio...
Chunk 1 não reconhecido
Chunk 2 não reconhecido
Chunk 3 não reconhecido
Chunk 4 não reconhecido
Chunk 5 não reconhecido
Chunk 6 não reconhecido
Chunk 7 não reconhecido
Chunk 8 não reconhecido
Chunk 9 não reconhecido
Chunk 10/236 transcrito
Chunk 11/236 transcrito
Chunk 12/236 transcrito
Chunk 13/236 transcrito
Chunk 14/236 transcrito
Chunk 15/236 transcrito
Chunk 16/236 transcrito
Chunk 17/236 transcrito
Chunk 18/236 transcrito
Chunk 19/236 transcrito
Chunk 20/236 transcrito
Chunk 21/236 transcrito
Chunk 22/236 transcrito
Chunk 23/236 transcrito
Chunk 24/236 transcrito
Chunk 25/236 transcrito
Chunk 26/236 transcrito
